# Chapter 6
## Spalyed and Partitioned Tables

### 6.1 Introduction
This chapter will cover the concept of splayed and partitioned tables, and also show how to load and save data using q. Both types of tables, especially partitioned, is useful for representing large data sets.

In [3]:
trades:([]code:`symbol$(); date:`date$(); time:`time$();price:`float$();volume:`short$();mktflag:`symbol$();comments:())

In [4]:
`trades upsert (
    (`CLM16; 2016.04.07; 10:20:00.329; 38.3; 4h; `E; "NYMEX/CME Group trade");
    (`GCM16; 2016.04.07; 12:00:00.055; 1239.7; 6h; `E; "");
    (`GCM16; 2016.04.07; 12:37:02.106; 1240.5; 1h; `E; "");
    (`CLM16; 2016.04.07; 13:00:00.128; 38.04; 3h; `E; "NYMEX/CME Group trade");
    (`VKX16; 2016.04.07; 13:22:05.617; 18.85; 5h; `E; "");
    (`GCM16; 2016.04.07; 14:35:01.241; 1241.2; 1h; `E; "");
    (`GCM16; 2016.04.08; 10:13:01.048; 1240f; 3h; `E; "");
    (`VKX16; 2016.04.08; 11:34:53.417; 18.53; 1h; `E; "Transaction represents a trade in two contract months in the same class");
    (`CLM16; 2016.04.08; 12:00:00.227; 40.61; 3h; `E; "NYMEX/CME Group trade");
    (`VXK16; 2016.04.08; 12:44:00.684; 18.44; 2h; `E; "Transaction represents a trade in two contract months in the same class");
    (`VXK16; 2016.04.08; 12:45:33.130; 18.49; 1h; `E; "Transaction represents a trade in two contract months in the same class");
    (`CLM16; 2016.04.08; 15:20:02.000; 40.78; 3h; `E; "NYMEX/CME Group trade");
    (`CLM16; 2016.04.11; 11:00:00.105; 41.43; 2h; `E; "NYMEX/CME Group trade");
    (`VXK16; 2016.04.11; 14:00:00.829; 18.35; 1h; `E; "");
    (`VXK16; 2016.04.11; 15:14:58.775; 19.05; 2h; `E; "");
    (`GCM16; 2016.04.11; 16:00:00.044; 1257.9; 1h; `E; "");
    (`GCM16; 2016.04.11; 16:28:34.311; 1258.7; 1h; `E; "")
    )

`trades


In [5]:
trades

code  date       time         price  volume mktflag comments                 ..
-----------------------------------------------------------------------------..
CLM16 2016.04.07 10:20:00.329 38.3   4      E       "NYMEX/CME Group trade"  ..
GCM16 2016.04.07 12:00:00.055 1239.7 6      E       ""                       ..
GCM16 2016.04.07 12:37:02.106 1240.5 1      E       ""                       ..
CLM16 2016.04.07 13:00:00.128 38.04  3      E       "NYMEX/CME Group trade"  ..
VKX16 2016.04.07 13:22:05.617 18.85  5      E       ""                       ..
GCM16 2016.04.07 14:35:01.241 1241.2 1      E       ""                       ..
GCM16 2016.04.08 10:13:01.048 1240   3      E       ""                       ..
VKX16 2016.04.08 11:34:53.417 18.53  1      E       "Transaction represents a..
CLM16 2016.04.08 12:00:00.227 40.61  3      E       "NYMEX/CME Group trade"  ..
VXK16 2016.04.08 12:44:00.684 18.44  2      E       "Transaction represents a..
VXK16 2016.04.08 12:45:33.130 18.49  1  

### 6.2 Saving a table as a single binary file

In [6]:
dataDir:"C:/Users/moazi/Documents/GitHub/Kdb-ML-Big-Data-Notes/test/db"
dataDir

"C:/Users/moazi/Documents/GitHub/Kdb-ML-Big-Data-Notes/test/db"


In [7]:
`dataDir set getenv`KDB_DATA

`dataDir


In [8]:
//`tradesfile set trades

Running the above line will save the trades table in a binary file in the current directory. I commented it out as to not cluter the directory, but comment it out and run to see.

In [8]:
(`$":",dataDir,"/tradesfile") set trades

[0;31m/tradesfile. OS reports: Access is denied.[0m: [0;31m/tradesfile. OS reports: Access is denied.[0m

The above line should save the file in the dedicated directroy I've created listed in dataDir.

Note: the brackets are needed around the directory path as a sym. This is due to right to left evaluaiton of kdb.

Now that we have the file saved, we can retrive the file into a table, using `get` or using `load`. 

In [8]:
trades1:get `$":",dataDir,"/tradesfile"
show trades1 ~ trades;
trades1

[0;31m/tradesfile. OS reports: The system cannot find the file specified.[0m: [0;31m/tradesfile. OS reports: The system cannot find the file specified.[0m

In [8]:
load `$":",dataDir,"/tradesfile";
tradesfile ~ trades;
tradesfile

[0;31m/tradesfile. OS reports: The system cannot find the file specified.[0m: [0;31m/tradesfile. OS reports: The system cannot find the file specified.[0m

The difference between get and laod, is load will load the file in with its given name "tradesfile", while get will get the data and hold it in whichever variable its assigned to.

Notice that the file discriptor, the code that denotes where in disk the file is, is writted as a symbol with ":" being the first character. The built-in funciton hsym exists to convert file paths to descriptors.

In [9]:
show `$dataDir;
show hsym `$dataDir

`
`


### 6.3 Splayed tables
Instead of saving as a binary file, we can also choose to save as a splayed table.

Splayed table saves each column of the data in a binary file. This is useful for storing medium sized (about 100million rows) tables with many columns. This is useful for optimal querying of large data and getting back results fast. It allows queries to only read in the files that's required and leave the rest on disk.

To save a table down as a spalyed table, you use the set function and add a "/" to the end to denote a directory. But that's not all


In [9]:
hsym[`$dataDir,"/tradesDir/"] set trades

[0;31mtype[0m: [0;31mtype[0m

The above failed due to the concept of enumeration for the type symbol.

In [10]:
meta trades

c       | t f a
--------| -----
code    | s    
date    | d    
time    | t    
price   | f    
volume  | h    
mktflag | s    
comments| C    


You can see that the columns code and mktflag is of type symbol, if we exclude those it'll save.

In [11]:
hsym[`$dataDir,"/tradesDir/"] set `code`mktflag _ trades

`:/tradesDir/


In [12]:
\dir test\db\tradesDir

" Volume in drive C is Windows-SSD"
" Volume Serial Number is 4244-28BF"
""
" Directory of C:\\Users\\moazi\\Documents\\GitHub\\Kdb-ML-Big-Data-Notes\\te..
""
"02/24/2022  07:03 AM    <DIR>          ."
"02/24/2022  07:45 AM    <DIR>          .."
"02/24/2022  07:03 AM                53 .d"
"02/24/2022  07:03 AM             4,232 code"
"02/24/2022  07:03 AM             4,232 comments"
"02/24/2022  07:03 AM             4,448 comments#"
"02/24/2022  07:03 AM                84 date"
"02/24/2022  07:03 AM             4,232 mktflag"
"02/24/2022  07:03 AM               152 price"
"02/24/2022  07:02 AM                34 sym"
"02/24/2022  07:03 AM                84 time"
"02/24/2022  07:03 AM                50 volume"
"              10 File(s)         17,601 bytes"
"               2 Dir(s)  781,979,492,352 bytes free"


The above shows the list of files saved when saved as a spalyed table. Besides the `.d` and `comments#` file the rest are associated with the columns of trades. We are missing code and mktflag because those were ommited. We can save those by enumeration. We can do that with the function `.Q.en`

In [13]:
tDir:hsym[`$dataDir,"/tradesDir/"];
tradesEnum:.Q.en[tDir;trades];  // enumerating

In [14]:
tDir set tradesEnum

`:/tradesDir/


In [15]:
\dir test\db\tradesDir

" Volume in drive C is Windows-SSD"
" Volume Serial Number is 4244-28BF"
""
" Directory of C:\\Users\\moazi\\Documents\\GitHub\\Kdb-ML-Big-Data-Notes\\te..
""
"02/24/2022  07:03 AM    <DIR>          ."
"02/24/2022  07:45 AM    <DIR>          .."
"02/24/2022  07:03 AM                53 .d"
"02/24/2022  07:03 AM             4,232 code"
"02/24/2022  07:03 AM             4,232 comments"
"02/24/2022  07:03 AM             4,448 comments#"
"02/24/2022  07:03 AM                84 date"
"02/24/2022  07:03 AM             4,232 mktflag"
"02/24/2022  07:03 AM               152 price"
"02/24/2022  07:02 AM                34 sym"
"02/24/2022  07:03 AM                84 time"
"02/24/2022  07:03 AM                50 volume"
"              10 File(s)         17,601 bytes"
"               2 Dir(s)  781,979,492,352 bytes free"


In [16]:
tradesEnum[`code]

`sym$`CLM16`GCM16`GCM16`CLM16`VKX16`GCM16`GCM16`VKX16`CLM16`VXK16`VXK16`CLM16..


In [17]:
trades[`code]

`CLM16`GCM16`GCM16`CLM16`VKX16`GCM16`GCM16`VKX16`CLM16`VXK16`VXK16`CLM16`CLM1..


Now the table is saved as a splyed table in full, with code and mktflag column.  Notice how the `code` column is enumerated in `tradesEnum`. We also have an additional `sym` file. This is for the enumeration values. When we have a column of type symbol, we save all the symbols in a sym file, and the columns of type symbol lists the index of the symbols occurance in that sym file. Thus making symbols much faster than string to query.

Note: Splayed table can't save keyed table. The table will first need to be unkeyed in order to save, else we'll recive a type error.

The files in the `tradesDir` directory contains the serealized data for each column the file is named after. You can also save individual columns as serialized files as well.

In [17]:
hsym[`$dataDir,"/myPrice"] set trades[`price];
myPrice:get hsym[`$dataDir,"/myPrice"];

[0;31m/myPrice. OS reports: Access is denied.[0m: [0;31m/myPrice. OS reports: Access is denied.[0m

In [17]:
myPrice ~ trades[`price]

[0;31mmyPrice[0m: [0;31mmyPrice[0m

##### .d file

The .d file is the collection of columns in the table set as a file.

In [18]:
cols trades

`code`date`time`price`volume`mktflag`comments


In [19]:
get hsym[`$dataDir,"/tradesDir/.d"]

`code`date`time`price`volume`mktflag`comments


Regarding the `comments` and `comments#` files. Whenever we save a column of type list down, in this case its a string column which is a list of charecters, it'll save in 2 files. The file `comments` here stores a serealized list of charecters, and the `comments#` file stores which row has how many charecters so it can unserialize back into the table. For example the first comment is "NYMEX/CME Group trade", so the `comments` file will hold each charecter and `comments#` will store it is the first 25 charecters.

To load a splayed table, we can use the function load, or load it in like a .q file with \l. Note when loading in (as opposed to saving) we don't include the trailing "/". We do so below with the vs function.

In [20]:
tDir

`:/tradesDir/


In [21]:
first ` vs tDir

`:/tradesDir


In [22]:
load first ` vs tDir

`tradesDir


In [23]:
tradesDir

code  date       time         price  volume mktflag comments                 ..
-----------------------------------------------------------------------------..
CLM16 2016.04.07 10:20:00.329 38.3   4      E       "NYMEX/CME Group trade"  ..
GCM16 2016.04.07 12:00:00.055 1239.7 6      E       ""                       ..
GCM16 2016.04.07 12:37:02.106 1240.5 1      E       ""                       ..
CLM16 2016.04.07 13:00:00.128 38.04  3      E       "NYMEX/CME Group trade"  ..
VKX16 2016.04.07 13:22:05.617 18.85  5      E       ""                       ..
GCM16 2016.04.07 14:35:01.241 1241.2 1      E       ""                       ..
GCM16 2016.04.08 10:13:01.048 1240   3      E       ""                       ..
VKX16 2016.04.08 11:34:53.417 18.53  1      E       "Transaction represents a..
CLM16 2016.04.08 12:00:00.227 40.61  3      E       "NYMEX/CME Group trade"  ..
VXK16 2016.04.08 12:44:00.684 18.44  2      E       "Transaction represents a..
VXK16 2016.04.08 12:45:33.130 18.49  1  

In [24]:
\l test\db\tradesDir

`test\db\tradesDir


In [25]:
tradesDir

code  date       time         price  volume mktflag comments                 ..
-----------------------------------------------------------------------------..
CLM16 2016.04.07 10:20:00.329 38.3   4      E       "NYMEX/CME Group trade"  ..
GCM16 2016.04.07 12:00:00.055 1239.7 6      E       ""                       ..
GCM16 2016.04.07 12:37:02.106 1240.5 1      E       ""                       ..
CLM16 2016.04.07 13:00:00.128 38.04  3      E       "NYMEX/CME Group trade"  ..
VKX16 2016.04.07 13:22:05.617 18.85  5      E       ""                       ..
GCM16 2016.04.07 14:35:01.241 1241.2 1      E       ""                       ..
GCM16 2016.04.08 10:13:01.048 1240   3      E       ""                       ..
VKX16 2016.04.08 11:34:53.417 18.53  1      E       "Transaction represents a..
CLM16 2016.04.08 12:00:00.227 40.61  3      E       "NYMEX/CME Group trade"  ..
VXK16 2016.04.08 12:44:00.684 18.44  2      E       "Transaction represents a..
VXK16 2016.04.08 12:45:33.130 18.49  1  

Finally, we list below two useful function to create and append a splayed table.

In [26]:
\l quantQ-master/lib/quantQ_io.q

In [27]:
.quantQ.io.saveSplayedTab

{[tabPath;pvar;table]
    // tabPath -- path where to store the table
    // pvar -- variable to sort and index on for fast querying
    // table -- name of the table to save
    @[;pvar;`p#] pvar xasc (` sv (tabPath;`;table;`)) set .Q.en[tabPath] get ..
 }


In [28]:
.quantQ.io.saveSplayedTab[hsym[`$dataDir,"/sdb"]; `code; `trades]

`:/sdb//trades/


In [29]:
\dir test\db\sdb

" Volume in drive C is Windows-SSD"
" Volume Serial Number is 4244-28BF"
""
" Directory of C:\\Users\\moazi\\Documents\\GitHub\\Kdb-ML-Big-Data-Notes\\te..
""
"02/24/2022  07:45 AM    <DIR>          ."
"02/24/2022  07:45 AM    <DIR>          .."
"02/24/2022  07:45 AM                34 sym"
"02/24/2022  07:45 AM    <DIR>          trades"
"               1 File(s)             34 bytes"
"               3 Dir(s)  781,979,492,352 bytes free"


In [30]:
\dir test\db\sdb\trades

" Volume in drive C is Windows-SSD"
" Volume Serial Number is 4244-28BF"
""
" Directory of C:\\Users\\moazi\\Documents\\GitHub\\Kdb-ML-Big-Data-Notes\\te..
""
"02/24/2022  07:45 AM    <DIR>          ."
"02/24/2022  07:45 AM    <DIR>          .."
"02/24/2022  07:45 AM                53 .d"
"02/24/2022  07:45 AM             4,416 code"
"02/24/2022  07:45 AM             4,232 comments"
"02/24/2022  07:45 AM             4,448 comments#"
"02/24/2022  07:45 AM                84 date"
"02/24/2022  07:45 AM             4,232 mktflag"
"02/24/2022  07:45 AM               152 price"
"02/24/2022  07:45 AM                84 time"
"02/24/2022  07:45 AM                50 volume"
"               9 File(s)         17,751 bytes"
"               2 Dir(s)  781,979,426,816 bytes free"


Above it saved trades in a db called "sdb" and placed the sym file above the splayed table so multiple splayed tables can share the sym file.

In [31]:
.quantQ.io.appendSplayedTab

{[tabPath;pvar;table;table2Add]
    // tabPath -- path where to store the table
    // pvar -- variable to sort and index on for fast querying
    // table -- name of the saved table
    // table2Add -- table to append
    @[;pvar;`p#] pvar xasc (` sv (tabPath;`;table;`)) upsert .Q.en[tabPath] t..
 }


This function exists to add new data to existing splayed table. When passing in the new data in table2Add, you pass in without a backtick. The two functions are almost identicle except the append uses upsert instead of save.

### 6.4 Partitioned Table
In addition to splaying table, we may partition tables. Partitions are defined by a partition column, for which each distinct value v a partition  is created. Typically this goes by date. So for each date a new patition is created, and in each partition a table or many tabels can be stored. Each table then is spalyed.

In [32]:
mydbdir:hsym `$dataDir,"/mydb";

{`trd set select from trades where date=x; .Q.dpft[mydbdir;x;`code;`trd]; delete trd from `.;} each distinct trades[`date]

::
::
::


In [35]:
mydbdir

`:/mydb


 The funciton above calls .Q.dfpt for each instance of the partitioned value

In [36]:
trades[`date]

2016.04.07 2016.04.07 2016.04.07 2016.04.07 2016.04.07 2016.04.07 2016.04.08 ..


In [37]:
select from trades where date=2016.04.07

code  date       time         price  volume mktflag comments               
---------------------------------------------------------------------------
CLM16 2016.04.07 10:20:00.329 38.3   4      E       "NYMEX/CME Group trade"
GCM16 2016.04.07 12:00:00.055 1239.7 6      E       ""                     
GCM16 2016.04.07 12:37:02.106 1240.5 1      E       ""                     
CLM16 2016.04.07 13:00:00.128 38.04  3      E       "NYMEX/CME Group trade"
VKX16 2016.04.07 13:22:05.617 18.85  5      E       ""                     
GCM16 2016.04.07 14:35:01.241 1241.2 1      E       ""                     
